In [27]:
import pandas as pd
import torch
import numpy as np

data = pd.read_csv('distillation-tower.csv')

X = data.drop('VapourPressure', axis=1)
X = X.iloc[:, 1:]
y = data['VapourPressure']

In [28]:
invt = data['InvTemp3']
vp = data['VapourPressure']
vp = np.log(vp)

In [29]:
from sklearn.linear_model import LinearRegression

invt = np.array(invt).reshape(-1, 1)
vp = np.array(vp)

model = LinearRegression()

model.fit(invt, vp)

vp_pred = np.exp(model.predict(invt))

In [30]:
# Calculate delta between experimentally measure vapor pressure and empirical model prediction
y -= vp_pred

In [31]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [32]:
from torch.utils.data import TensorDataset, DataLoader

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [33]:
import torch.nn as nn

class VaporPressureNet(nn.Module):
    def __init__(self, dropout_rate=0.1):
        super(VaporPressureNet, self).__init__()
        self.fc1 = nn.Linear(X.shape[1], 64)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(64, 64)
        # self.dropout2 = nn.Dropout(dropout_rate)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu(x)
        # x = self.dropout2(x)
        x = self.fc3(x)
        return x
# import torch.nn as nn
# 
# class VaporPressureNet(nn.Module):
#     def __init__(self):
#         super(VaporPressureNet, self).__init__()
#         self.fc1 = nn.Linear(X.shape[1], 64) 
#         self.relu = nn.ReLU()
#         self.fc2 = nn.Linear(64, 64)
#         self.fc3 = nn.Linear(64, 1) 
# 
#     def forward(self, x):
#         x = torch.relu(self.fc1(x))
#         x = torch.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x

In [34]:
model = VaporPressureNet()

criterion = nn.MSELoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 3000

for epoch in range(epochs):
    model.train() 
    running_loss = 0.0
    for inputs, targets in train_dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_dataloader.dataset)
    print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}')

model.eval() 
with torch.no_grad(): 
    predictions = []
    targets = []
    for inputs, true_values in test_dataloader:
        outputs = model(inputs)
        predictions.extend(outputs.squeeze().tolist())
        targets.extend(true_values.tolist())

from sklearn.metrics import r2_score, mean_absolute_error

r2 = r2_score(targets, predictions)
mae = mean_absolute_error(targets, predictions)
print(f'R^2 Score on test set: {r2:.4f}')
print(f'Mean absolute error on test set: {mae:.4f}')

Epoch 1/3000, Loss: 409.2823
Epoch 2/3000, Loss: 109.2754
Epoch 3/3000, Loss: 91.5075
Epoch 4/3000, Loss: 55.1661
Epoch 5/3000, Loss: 42.5549
Epoch 6/3000, Loss: 29.0340
Epoch 7/3000, Loss: 26.1772
Epoch 8/3000, Loss: 23.7184
Epoch 9/3000, Loss: 21.2613
Epoch 10/3000, Loss: 15.5065
Epoch 11/3000, Loss: 15.3872
Epoch 12/3000, Loss: 13.7621
Epoch 13/3000, Loss: 14.4008
Epoch 14/3000, Loss: 10.3122
Epoch 15/3000, Loss: 10.6940
Epoch 16/3000, Loss: 8.4916
Epoch 17/3000, Loss: 10.0596
Epoch 18/3000, Loss: 8.4661
Epoch 19/3000, Loss: 8.2160
Epoch 20/3000, Loss: 7.8252
Epoch 21/3000, Loss: 7.2241
Epoch 22/3000, Loss: 7.4855
Epoch 23/3000, Loss: 8.3698
Epoch 24/3000, Loss: 7.8402
Epoch 25/3000, Loss: 8.1248
Epoch 26/3000, Loss: 7.8374
Epoch 27/3000, Loss: 8.7969
Epoch 28/3000, Loss: 8.0150
Epoch 29/3000, Loss: 7.1123
Epoch 30/3000, Loss: 7.3250
Epoch 31/3000, Loss: 7.1259
Epoch 32/3000, Loss: 6.6014
Epoch 33/3000, Loss: 6.6882
Epoch 34/3000, Loss: 7.1829
Epoch 35/3000, Loss: 6.0990
Epoch 36/30

In [35]:
def evaluate(n):
    single_observation = X_test.iloc[n]
    single_observation = single_observation.apply(pd.to_numeric, errors='coerce').fillna(0)
    single_observation_tensor = torch.tensor(single_observation.values, dtype=torch.float32).unsqueeze(0)
    model.eval()
    with torch.no_grad():
        prediction = model(single_observation_tensor)     
    # lnp = vp_pred[80]
    vapor = vp_pred[n]
    y = prediction.item() + vapor
    print("Predicted Vapour Pressure:", prediction.item())
    print("Experimental Vapour pressure:", y_test.iloc[n])
    print(y)


In [36]:
evaluate(30)

Predicted Vapour Pressure: -0.5054162740707397
Experimental Vapour pressure: -1.3230514331946495
30.824173222961225
